<a href="https://colab.research.google.com/github/filippozuddas/ML-SRT-SETI/blob/main/notebooks/train_CVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install setigen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.2.3
    Uninstalling pyparsing-3.2.3:
      Successfully uninstalled pyparsing-3.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydot 3.0.4 requires pyparsing>=3.0.9, but you have pyparsing 2.4.7 which is incompatible.


In [2]:
import sys
import os

# --- CONFIGURAZIONE PERCORSI ---
PROJECT_ROOT = "/content/filippo/ML-SRT-SETI"
PROJECT_ROOT_LOCAL = "/home/filippo/TirocinioSETI/ML-SRT-SETI"

if not os.path.exists(PROJECT_ROOT_LOCAL):
    raise FileNotFoundError(f"{PROJECT_ROOT_LOCAL} folder not found")

os.chdir(PROJECT_ROOT_LOCAL)

if PROJECT_ROOT_LOCAL not in sys.path:
    sys.path.append(PROJECT_ROOT_LOCAL)

print(f"✅ Setup completed!")
print(f"   Working directory: {os.getcwd()}")

✅ Setup completed!
   Working directory: /home/filippo/TirocinioSETI/ML-SRT-SETI


In [3]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from src import config, models

# Verifica GPU
print(f"TensorFlow Version: {tf.__version__}")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-11-17 17:14:16.106203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-17 17:14:16.650519: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 17:14:16.724509: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-11-17 17:14:16.724620: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

TensorFlow Version: 2.10.1
Num GPUs Available:  0


2025-11-17 17:14:23.898686: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-11-17 17:14:23.899283: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (filippo-ideapad-slim-5): /proc/driver/nvidia/version does not exist


In [ ]:
# Percorso del file generato nello step precedente
DATA_PATH = "data/processed/train_class0_5k.npy"

# 1. Carica i dati
print(f"Caricamento dati da {DATA_PATH}...")
dataset_cadences = np.load(DATA_PATH)
print(f"Shape originale (Cadenze): {dataset_cadences.shape}")

# 2. Reshape: Da (N_samples, 6, 16, 512, 1) a (N_samples * 6, 16, 512, 1)
# Fondiamo la dimensione delle cadenze con quella dei campioni
# -1 dice a numpy di calcolare automaticamente la dimensione
X_all = dataset_cadences.reshape(-1, config.TCHANS, config.FCHANS_FINAL, 1)

print(f"Shape per il training (Frames singoli): {X_all.shape}")

# 3. Train/Test Split
# Usiamo il random_state per riproducibilità
X_train, X_val = train_test_split(X_all, test_size=0.2, random_state=42)

print(f"Training frames: {X_train.shape}")
print(f"Validation frames: {X_val.shape}")

Caricamento dati da data/processed/train_class0_5k.npy...


In [14]:
print("Verifica integrità dati...")
print(f"Min value: {np.min(X_train)}")
print(f"Max value: {np.max(X_train)}")
print(f"Contains NaNs: {np.isnan(X_train).any()}")
print(f"Contains Infs: {np.isinf(X_train).any()}")

if np.isnan(X_train).any() or np.isinf(X_train).any():
    print("⚠️ ATTENZIONE: Il dataset contiene valori non validi! Il problema è nel preprocessing.")
else:
    print("✅ Dati puliti. Il problema è nell'architettura o nei parametri.")

Verifica integrità dati...
Min value: 0.0
Max value: 1.0
Contains NaNs: False
Contains Infs: False
✅ Dati puliti. Il problema è nell'architettura o nei parametri.


In [1]:
# MirroredStrategy copia il modello su tutte le GPU disponibile e divide il batch
strategy = tf.distribute.MirroredStrategy()

# Parametri
input_shape = (config.TCHANS, config.FCHANS_FINAL, 1)
latent_dim = config.LATENT_DIM # 8


with strategy.scope():
  # Costruzione Encoder e Decoder
  encoder = models.build_encoder(input_shape, latent_dim)
  decoder = models.build_decoder(latent_dim, target_shape=input_shape)

  # 2. CVAE Completo
  # beta=1.0 è un VAE standard.
  # beta > 1 forza una rappresentazione latente più "districata" (disentangled)
  cvae = models.CVAE(encoder, decoder, beta=1)

  # Compile
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
  cvae.compile(optimizer=optimizer)

# (Opzionale) Visualizza sommario encoder
encoder.summary()

NameError: name 'tf' is not defined

In [19]:
# Configurazione Training
EPOCHS = 20
BATCH_SIZE = config.BATCH_SIZE # 32

print("Avvio training...")

history = cvae.fit(
    X_train,
    validation_data=(X_val, X_val), # Nel VAE input e target sono uguali (ricostruzione)
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True
)

print("Training completato.")

Avvio training...
Epoch 1/20


2025-11-17 15:33:20.238495: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


750/750 [==============================] - ETA: 0s - total_loss: 4391.9995 - reconstruction_loss: 4377.3174 - kl_loss: 14.6829

2025-11-17 15:37:39.546036: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


750/750 [==============================] - 284s 374ms/step - total_loss: 4391.9995 - reconstruction_loss: 4377.3174 - kl_loss: 14.6829 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00 - val_val_total_loss: 4281.4121 - val_val_reconstruction_loss: 4272.2764 - val_val_kl_loss: 9.1371
Epoch 2/20
699/750 [==========================>...] - ETA: 16s - total_loss: 4270.0732 - reconstruction_loss: 4261.9990 - kl_loss: 8.0724

KeyboardInterrupt: 

In [ ]:
import os
import datetime
import tensorflow as tf
from tensorflow import keras
from src.generator import SetigenGenerator
from src.modelsv2 import SETI_VAE

# --- CONFIGURAZIONE ---
# Hardware & Percorsi
GPUS = tf.config.list_physical_devices('GPU')
LOG_DIR = "logs"
CHECKPOINT_DIR = "checkpoints"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Iperparametri
# Batch size globale: con 2x4090 possiamo spingere.
# Inizia con 64 o 128. Se hai OOM (Out of Memory), riduci.
BATCH_SIZE_PER_REPLICA = 64 
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * len(GPUS) if len(GPUS) > 0 else BATCH_SIZE_PER_REPLICA

EPOCHS = 100
STEPS_PER_EPOCH = 1000  # Quanti batch processare per considerare finita un'epoca
LEARNING_RATE = 1e-4    # Tasso di apprendimento iniziale (Paper: 0.0005)
LATENT_DIM = 8          # Dimensione spazio latente (Paper: 8)

# Parametri Generatore (Simulazione)
SNR_RANGE = (10, 50)
DRIFT_RANGE = (0.5, 8.0)

def main():
    print(f"🚀 Avvio Training SETI VAE")
    print(f"Num GPUs Available: {len(GPUS)}")
    print(f"Global Batch Size: {GLOBAL_BATCH_SIZE}")
    
    # 1. Setup Strategia Multi-GPU
    strategy = tf.distribute.MirroredStrategy()
    print(f'Number of devices: {strategy.num_replicas_in_sync}')

    # 2. Preparazione Dati
    # Creiamo due generatori: uno per training e uno per validation
    # (Nota: essendo dati sintetici, la validation serve a controllare che la loss scenda su dati nuovi)
    
    train_gen = SetigenGenerator(
        batch_size=GLOBAL_BATCH_SIZE,
        snr_range=SNR_RANGE,
        drift_range=DRIFT_RANGE
    )
    train_dataset = train_gen.get_dataset()

    # Dataset di Validazione (Separiamo per monitorare l'overfitting)
    val_gen = SetigenGenerator(
        batch_size=GLOBAL_BATCH_SIZE,
        snr_range=SNR_RANGE, 
        drift_range=DRIFT_RANGE
    )
    val_dataset = val_gen.get_dataset()

    # 3. Costruzione Modello nello Scope della Strategia
    with strategy.scope():
        # Istanziamo il modello VAE
        # Input shape: (6, 16, 512, 1) - La cadenza intera
        vae = SETI_VAE(
            input_shape=(6, 16, 512, 1),
            latent_dim=LATENT_DIM,
            beta=1.5,  # Peso KL
            alpha=10.0 # Peso Clustering
        )
        
        # Compiliamo il modello
        # Non passiamo una loss function qui perché è gestita internamente nel train_step del modello custom
        vae.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE))

    # 4. Callbacks
    # Nome univoco per il run
    run_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    # Salva i pesi migliori
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"vae_seti_{run_name}_best.h5")
    checkpoint_cb = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_best_only=True,
        save_weights_only=True, # Salviamo solo i pesi per flessibilità (custom model)
        monitor='val_total_loss', # Monitoriamo la loss totale validazione
        mode='min',
        verbose=1
    )
    
    # Log CSV per grafici
    csv_logger = keras.callbacks.CSVLogger(os.path.join(LOG_DIR, f"training_log_{run_name}.csv"))
    
    # Riduzione LR se ci blocchiamo
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor='val_total_loss', 
        factor=0.5, 
        patience=5, 
        min_lr=1e-6,
        verbose=1
    )

    # Early Stopping
    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_total_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )

    # 5. Avvio Training
    print("\nDataset e Modello pronti. Inizio fit()...")
    
    history = vae.fit(
        train_dataset,
        epochs=EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=val_dataset,
        validation_steps=100, # 100 batch di validazione
        callbacks=[checkpoint_cb, csv_logger, reduce_lr, early_stop]
    )

    print("✅ Training completato.")
    
    # Salvataggio finale (modello completo se possibile, o pesi)
    final_save_path = os.path.join(CHECKPOINT_DIR, f"vae_seti_{run_name}_final_weights.h5")
    vae.save_weights(final_save_path)
    print(f"Pesi finali salvati in: {final_save_path}")

if __name__ == "__main__":
    main()